# Rozwiązanie Lab 4

In [1]:
import numpy as np
from scipy import sparse

In [2]:
with open("lines.txt", "r", encoding="UTF-8") as f1:
    unprocessed_e_mails = f1.readlines()

#### Klasa EMAIL
Przechowuje informacje o jednym mailu z pliku lines
self.email - treść maila
self.word_dict - reprezentacja maila bag_of_words, można to utożsamiać z wektorem, lecz jest to dict
self.index_vector - przechowuje id słów, które występują w tym tekście, tej reprezentacji używam przy metrykach dice, lcs, levenshtein
self.vector_norm - norma euklidesowa wektora

In [3]:
class EMAIL:
    def __init__(self, email: str, word_dict: dict[str, int]):
        self.email: str = email
        self.word_dict : dict[str, int] = word_dict
        self.index_vector = None
        self.vector_norm = 1

Processing tekstu

In [4]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from string import punctuation

def process_text(email: str) -> EMAIL:
    text = "".join(list(map(lambda c: " " if c in punctuation else c, email)))
    words = [word.lower() for word in word_tokenize(text)]
    return EMAIL(email, dict(FreqDist(words)))

In [5]:
emails = list(map(process_text, unprocessed_e_mails))

Tworzenie alfabetu (biorę wszystkie słowa jakie występują w tekstach)

In [6]:
alphabet = {}
count = {}
i = 0
for e in emails:
    for word in e.word_dict:
        if word not in alphabet:
            alphabet[word] = i
            count[word] = 1
            i += 1
        else:
            count[word] += 1
M1 = len(alphabet)
N1 = len(emails)
print(M1)
print(N1)

17804
6751


In [7]:
for e in emails:
    indices = list(map(lambda x: alphabet[x], e.word_dict.keys()))
    e.index_vector = np.sort(np.array(indices))

Tworzenie macierzy, w której każda kolumna reprezentuje jakieś słowo jako wektor. Oczywiście jest to macierz rzadka.
Argument words jest to alfabet słów, które występują w mailach w liście mails. M to rozmiar alfabetu a N to ilość maili.
Argument normalize umożliwia stworzenie macierzy w której wektory są znormalizowane (dzięki temu można szybko obliczyć wartości metryki cosinusowej)

In [32]:
def create_sparse_matrix(mails: list[EMAIL], words: dict[str, int], M: int, N: int, normalize=False):
    nonzero_entries = sum(len(e.index_vector) for e in mails)
    data = np.zeros(nonzero_entries)
    row = np.copy(data)
    col = np.copy(data)
    ind = 0
    for i, e in enumerate(mails):
        start = ind
        for w in e.word_dict:
            data[ind] = e.word_dict[w]
            row[ind] = words[w]
            col[ind] = i
            ind += 1
        length = np.linalg.norm(data[start:ind])
        e.vector_norm = length
        if normalize:
            data[start:ind] /= length

    return sparse.csr_matrix((data, (row, col)), shape=(M, N), dtype=float)

In [9]:
bag_of_words = create_sparse_matrix(emails, alphabet, M1, N1)
bag_of_words_n = create_sparse_matrix(emails, alphabet, M1, N1, normalize=True)

## Implementacja metryk

Przyjmuje, że czym jakieś dwa dokumenty są bliżej, to znaczy ,że wartość metryki jest bliżej zera. Dlatego od wartość cosinusa odejmuje od zera

### Metryka cosinusowa
Od razu obliczam macierz która zawiera wartość metryki dla każdej pary tekstów. Wyjściowo macierz ma wymiary N x N, gdzie N to liczba tekstów. Obliczanie wartości metryk za każdym razem dla jakiś dwóch teksów zajmowało, bardzo długo, lecz wykorzystując mnożenie macierzy rzadkich, można wartości tych metryk obliczyć bardzo szybko.
Podana macierz musi być już znormalizowana.

In [65]:
def cosine(matrix):
    return 1 - np.array((matrix.T @ matrix).todense())

### Metryka euklidesowa
Postępuje analogicznie jak w przypadku metryki cosinusowej i obliczam wartości metryki korzystając z działań na macierzach.

In [11]:
def euclidian(matrix, e_mails, N):
    X = np.array([e.vector_norm**2 for e in e_mails]).reshape((N, 1)) @ np.ones(N).reshape((1, N))
    return np.sqrt(np.abs(X + X.T - 2*np.array((matrix.T @ matrix).todense())))

Dla pozostałych metryk nie da się specjalnie przyśpieszyć obliczania ich wartości. Klasteryzacje z użyciem tych metryk będę przeprowadzał na dużo mniejszym zbiorze tekstów

### Najdłuższy wspólny podciąg

In [66]:
def lcs(text1: EMAIL, text2: EMAIL) -> float:
    seq_a = text1.index_vector
    seq_b = text2.index_vector
    m = len(seq_a)
    n = len(seq_b)
    dp = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if seq_a[i-1] == seq_b[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    return 1 - (dp[-1][-1] / max(m, n))

### Odległość edycyjna

In [68]:
def levenshtein(text1: EMAIL, text2: EMAIL) -> float:
    seq_a = text1.index_vector
    seq_b = text2.index_vector
    m = len(seq_a)
    n = len(seq_b)
    dp = [[float("inf") for _ in range(n + 1)] for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for i in range(n + 1):
        dp[0][i] = i

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if seq_a[i-1] == seq_b[j - 1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i-1][j-1] + 1, dp[i-1][j] + 1, dp[i][j - 1] + 1)

    return dp[-1][-1] / max(m, n)

### Współczynnik DICE

In [69]:
def dice(text1: EMAIL, text2: EMAIL) -> float:
    seq_a = text1.index_vector
    seq_b = text2.index_vector
    count_union = len([x for x in seq_a if x in seq_b])
    count_1 = len(seq_a)
    count_2 = len(seq_b)
    return 1 - (2*count_union / (count_1 + count_2))

## Klasteryzacja

Do klasteryzacji używam algorytmu k-means. Jest on jednak trochę zmodyfikowany. W oryginalnym algorytmie na początku losowo wybierało się k przedstawicieli grup. Każdy tekst trafiał do grupy do której przedstawiciela miał najbliżej według danej metryki. Następnie kolejnych przedstawicieli się już nie losuje, tylko bierze się średnią z wartości w danym klasterze. Jednak implementacja takiego trudna byłaby trudna, gdyż wtedy pojawiałyby się nowe punkty, a reprezentacja tekstów jako wielowymiarów wektórów, byłaby wymagająca obliczeniowo i prawdopodobnie algorytm nie policzyłby się dla całego zbioru maili. Poza tym pojęcie średniej zmienia zależnie od tego jakiej użyje się metryki. Ponadto przy metryce lcs, dice i levenshtein'a niemożliwe byłoby porównanie tekstów do wektora który powstał ze średniej. Dlatego jako średnią z klustera wybieram po prostu tekst, który znajduje się tym zbiorze i minimalizuje wariancje w obrębie tego klustera. Do oceny klasteryzacji używam indeksu Davies-Bouldine'a

In [72]:
%%time
cosine_metric = cosine(bag_of_words_n)
euclidian_metric = euclidian(bag_of_words, emails, N1)
print(cosine_metric.shape)
print(euclidian_metric.shape)

(6751, 6751)
(6751, 6751)
CPU times: total: 4.8 s
Wall time: 5.78 s


In [16]:
%%time
dice_metric = np.array([[dice(mail1, mail2) for mail1 in emails[:200]] for mail2 in emails[:200]])
lcs_metric = np.array([[lcs(mail1, mail2) for mail1 in emails[:200]] for mail2 in emails[:200]])
edit_metric = np.array([[levenshtein(mail1, mail2) for mail1 in emails[:200]] for mail2 in emails[:200]])

CPU times: total: 17.7 s
Wall time: 20.6 s


In [17]:
def find_centroid(cluster, metric):
    dists = metric[np.ix_(cluster, cluster)]
    variances = np.power(np.sum(dists, axis=1), 2)
    return cluster[np.argmin(variances)], np.min(variances)


def k_means_cluster(mails, metric=euclidian_metric, max_iter=100, k=10):
    n = metric.shape[0]
    a = np.arange(n)
    np.random.shuffle(a)
    cluster_means = a[:k]
    best_clustering = cluster_means
    best_variance = float("inf")

    for i in range(max_iter):
        clusters = [[cluster_means[i]] for i in range(k)]
        for i, mail in enumerate(mails):
            if i not in cluster_means:
                clusters[np.argmin(metric[i, cluster_means])].append(i)

        cluster_variance = 0
        prev_clustering = cluster_means.copy()

        for i in range(k):
            centroid, min_var = find_centroid(clusters[i], metric)
            cluster_means[i] = centroid
            cluster_variance += min_var

        if cluster_variance < best_variance:
            best_variance = cluster_variance
            best_clustering = cluster_means.copy()

        if np.all(prev_clustering == cluster_means):
            break

    cluster_means = best_clustering
    clusters = [[cluster_means[i]] for i in range(k)]
    for i, mail in enumerate(mails):
        if i not in cluster_means:
            clusters[np.argmin(metric[i, cluster_means])].append(i)

    return clusters


In [70]:
def davies_bouldin_index(clusters, metric):
    k = len(clusters)
    dispersion = []
    centroids = []
    for i in range(k):
        centroid, min_var = find_centroid(clusters[i], metric)
        dispersion.append(min_var / len(clusters[i]))
        centroids.append(centroid)

    separation = metric[np.ix_(centroids, centroids)]
    eps = pow(10, -3)
    separation[np.abs(separation) < eps] = 1000
    np.fill_diagonal(separation, 1000)
    similarity = np.array([[dispersion[i] + dispersion[j] for i in range(k)] for j in range(k)]) / separation

    return np.sum(np.max(similarity, axis=1)) / k

In [86]:
from copy import deepcopy

def find_best_clusters(mails, k_range, metric):
    best = float("inf")
    best_clusters = []
    index_val = []
    for i in k_range:
        clusters_ = k_means_cluster(mails, metric=metric, k=i)
        index_val.append(davies_bouldin_index(clusters_, metric))
        if index_val[-1] < best:
            best = index_val[-1]
            best_clusters = deepcopy(clusters_)
    return best_clusters

In [79]:
def print_clusters_parameters(mails, metric_type, clusters):
    print(f"Użyta metryka {metric_type}")
    print(f"Łącznie jest {len(clusters)} grup maili")
    print(f"Największy kluster składa się z {max(len(x) for x in clusters)} maili")
    print(f"Jest {len(list(filter(lambda x: len(x) == 1, clusters)))} grup jednoelementowych")
    print("Kilka przykładowych klustrów")
    print()

    a = np.arange(len(clusters))
    np.random.shuffle(a)
    for i in a[:20]:
        print("############################")
        for m in clusters[i]:
            print(mails[m].email)
        print()


In [47]:
%%time
b_clusters = find_best_clusters(emails, range(200, 3000, 100), euclidian_metric)

CPU times: total: 1min 4s
Wall time: 1min 5s


In [49]:
print_clusters_parameters(emails, "Euklidesowa", b_clusters)

Użyta metryka Euklidesowa
Łącznie jest 2700 grup maili
Największy kluster składa się z 227 maili
Jest 1496 grup jednoelementowych
Kilka przykładowych klustrów

############################
M+R FORWARDING (CHINA) LTD. GUANGZHOU BRANCH BLOCK D,20/FL.,GAOSHENG BLDG.,NO.109. TIYU ROAD.,WEST,GUANGZHOU,P.R.CHINA POSTAL CODE:510620


############################
MASHIMPORT LLC        B5,40,  BOLSHAYA SEMENOVSKAYA STR, 107023, MOSCOW, RUSSIA

MASHIMPORT LLC B5,40,  BOLSHAYA SEMENOVSKAYA STR, 107023, MOSCOW, RUSSIA

MASHIMPORT LLC  B5,40,  BOLSHAYA SEMENOVSKAYA STR, 107023, MOSCOW, RUSSIA

MASHIMPORT LLC      B5,40,  BOLSHAYA SEMENOVSKAYA STR, 107023, MOSCOW,RUSSIA

MASHIMPORT LLC B5,40,BOLSHAYA SEMENOVSKAYA STR.,107023,MOSCOW,RUSSIA


############################
ZHONGSHAN GALLOP INTERNATIONAL CO.,LTD NO 2,HAIBANG ROAD XIAOLAN ZHONGSHAN GUANGDONG CHINA

BEIJING KANG JIE KONG INTO CARGO AGENT CO LTD - ZHONGSHAN BRANCH UNIT702,SOUTH TOWER HOLIDAY PLAZA NO.6,XINGZHONG ROAD, ZHONGSHAN CHINA, PEOPL

In [87]:
%%time
b_clusters = find_best_clusters(emails, range(200, 3000, 100), cosine_metric)

CPU times: total: 2min 56s
Wall time: 2min 59s


In [88]:
print_clusters_parameters(emails, "Cosinusowa", b_clusters)

Użyta metryka Cosinusowa
Łącznie jest 2600 grup maili
Największy kluster składa się z 28 maili
Jest 1017 grup jednoelementowych
Kilka przykładowych klustrów

############################
JINHU GOLDEN SUN ENTERPRISE CO.,LTD. NO.22 JIANKANG ROAD,JINHU,JIANGSU,CHINA

JIANGSU GOLDEN ISLAND INTERNATIONALCO.,LTD. NO.217,QINGSHUITING WESTROAD, JIANGNING ECONMIC DEVELOPMENT+

JINHU GOLDEN SUN ENTERPRISE CO. ,LTD. NO.22 JIANKANG ROAD, JINHU ,JIANGSU, CHINA


############################
LLC "LENTA" SAVUSHKINA STREET 112 ST.PETERSBURG 197374 RUSSIAN FEDERATION


############################
FMG SHIPPING AND FORWARDING LTD., IMPORT@FMGSHIPPING.SPB.RU, 190020,ST. PETERSBURG RUSSIA, BUMAZHNAYA STR., 18 BUSINESS CENTER PORTAL, OF. 310 TEL.+7(812)4488283 FAX.+7(812)44


############################
PANALPINA CIS HELSINKI OY REPRESENTATIVE OFFICE BUSINESS-CENTRE "NOVOSPASSKIY DVOR", 7 BLOCK 9, DERBENEVSKAYA NABEREJNAYA 115114 MOSCOW RUSSIA PHONE +7 495 96 1 2559 FAX 2569


############################

In [55]:
%%time
b_clusters = find_best_clusters(emails[:200], range(20, 100, 5), dice_metric)

CPU times: total: 250 ms
Wall time: 255 ms


In [56]:
print_clusters_parameters(emails, "Dice", b_clusters)

Użyta metryka Dice
Łącznie jest 75 grup maili
Największy kluster składa się z 8 maili
Jest 26 grup jednoelementowych
Kilka przykładowych klustrów

############################
"AVANPORT"  LLC INN: 7839413675 KPP: 783901001 190020, SAINT PETERSBURG, QUAY OF THE OBVODNY CHANNEL, 134-136-138, BUILD. 101, LIT. A"

"ALLIANCE-TRADE" LLC INN: 7816391055 / KPP: 784601001 190020, Saint Petersburg, quay of the Obvodny channel, 138, bulk 1, liter.B

"AVANPORT"  LLC INN: 7839413675 KPP: 783901001 190020, SAINT PETERSBURG, QUAY OF  THE OBVODNY CHANNEL,134-136-138,  BUILD. 101, LIT. A"


############################
"ELECTROGROUP" (OOO),190068.RUSSIA,SAINT-PETERSBURG,UL.BOLSHAYA PODYACHESKAYA,5,LIT.A,POM.4-N

"ELECTROGROUP" (OOO),190068.RUSSIA,SAINT-PETERSBURG,UL.BOLSHAYA  PODYACHESKAYA,5,LIT.A,POM.4-N

"OOO" TECHNOLINE" 191123,ST.-PETERSBURG,UL. FURSCHTATSKAYA 48,LITER A"

<TORGOVIY SOYUZ>LENSOVETA STR., 16, LIT. A, APP.  4-N SAINT-PETERSBURG, 196135, RUSSIA


############################
"FT-LINE"

In [58]:
%%time
b_clusters = find_best_clusters(emails[:200], range(20, 100, 5), edit_metric)

CPU times: total: 234 ms
Wall time: 280 ms


In [59]:
print_clusters_parameters(emails, "Levenshtein", b_clusters)

Użyta metryka Levenshtein
Łącznie jest 85 grup maili
Największy kluster składa się z 8 maili
Jest 35 grup jednoelementowych
Kilka przykładowych klustrów

############################
"SUNTIRE" CO. LTD. TEL.: + 7 (812)458 54 38 E-MAIL: CLIENT@S-LOGISTIC.RU

/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA

"SUNTIRE"CO.LTD TEL:+7(812)458 54 38 E-MAIL:CLIENT@S-LOGISTIC.RU


############################
"KM" LTD 197183 RUSSIA,ST. PETERSBURG,SABIROVSKAYA STR.,50,OFF,115 OLEG TEREKHOV ZIP CODE:197183 PHONE:7911239-3109  FAX:7911239-3109


############################
"FILLOGISTIK" PP ZHOVKIVSKA 22 STREET 79019 LVOV,UKRAINE TEL:+380322458030 FAX:+380322458030


############################
1.MCT

1. MCT


############################
"TEHIMEKS"LLC 107497, BAIKALSKAYA STR., 1/3, BLD.1, MOSCOW, RUSSIA TEL(495)745-88-88 FAX:(495)380-01-24


############################
"SAME AS CONSIGNEE"


############################
"OOO" TECHNOLINE" 

In [60]:
%%time
b_clusters = find_best_clusters(emails[:200], range(20, 100, 5), lcs_metric)

CPU times: total: 250 ms
Wall time: 234 ms


In [61]:
print_clusters_parameters(emails, "LCS", b_clusters)

Użyta metryka LCS
Łącznie jest 85 grup maili
Największy kluster składa się z 8 maili
Jest 38 grup jednoelementowych
Kilka przykładowych klustrów

############################
"SUNTIRE"CO.LTD M.BALKANSKAYA STREET,58,LIT.A 192289,ST.PETERSBURG,RUSSIA

"INTERLINK LTD SP ZOO UI.  POMORSKA50A 81-314 GDYNIA BIELSKA@INTERLINK.BIZ.PL SIEJKOWSKA@INTERLINK.BIZ.PL...+++

"SUNTIRE" CO. LTD, M. BALKANSKAYA STREET, 58, LIT.A 192289, ST.-PETERSBURG, RUSSIA


############################
1.FMG SHIPPING AND FORWARDING LTD 190020, SAINT PETERSBURG, LIFLYANDSKAYA STR.,6,LITERA "A",BUILDING 31H,OFFICE 310 POST CODE:190020 TEL:+7812 448 82 83+

"FMG SHIPPING AND FORWARDING, LTD."190020, SAINT PETERSBURG,LIFLYANDSKAYA STR., 6,LITERA "A",BUILDING 31H,OFFICE 310 POST CODE:190020 TEL:+7 812 448 82 83 FAX:+7 812 448 04 9

"TVA" DUNTES STR 23A,OFFICE 301, LV1005 R IGA, LATVIA RIGA RIX 36309 LATVIA T: +371 (6) 770-9141 F: + 371(6) 738-2375

1/FMG SHIPPING AND FORWARDING LTD 190020, SAINT PETERSBURG, LIFLYANDSKAYA

Czas obliczeń dla metryk dice, levehnstain i lcs był bardzo krótki, gdyż macierze były już obliczone, a samo przygotowanie macierzy 200 x 200 dla tych metryk trwało już po 5 sekund. Lepsza okazała się także metryka cosinusowa. Algorytm z użyciem tej metryki liczył się około 3 razy dłużej niż metryka euklidesowa. Przy obliczonych macierzach wykonywały na jedną iterację miej więcej tyle samo operacji, lecz warunkiem przerwania algorytmu było albo osiągięcie maksymalnej liczby iteracji lub powtórzenie się wektora tekstów będących średnimi z grup. Widać, że w metryce cosinusowej dokonywało się dużo więcej zmian podczas liczenia i liczność największej grupy jest dużo mniejsza niż po obliczeniach w metryce euklidesowej.

## Preprocessing alfabetu

Zostawiam tylko słowa, które wystąpiły przynajmiej dwa razy i usuwam dokumenty, które przez to nie mają żadnego słowa. ( był dokładnie jeden)

In [94]:
alphabet2 = {}
ind = 0
for word in alphabet:
    if count[word] > 1:
        alphabet2[word] = ind
        ind += 1

In [95]:
def remove_words(mail: EMAIL):
    new_word_dict = {}
    for w in mail.word_dict:
        if w in alphabet2:
            new_word_dict[w] = mail.word_dict[w]
    if len(new_word_dict) == 0:
        return None
    return EMAIL(mail.email, new_word_dict)

In [96]:
emails2 = list(filter(lambda x: x is not None, map(remove_words, emails)))
for e in emails2:
    indices = list(map(lambda x: alphabet2[x], e.word_dict.keys()))
    e.index_vector = np.sort(np.array(indices))

M2 = len(alphabet2)
N2 = len(emails2)
print(M2)
print(N2)

7224
6750


In [97]:
bag_of_words2 = create_sparse_matrix(emails2, alphabet2, M2, N2)
bag_of_words_n2 = create_sparse_matrix(emails2, alphabet2, M2, N2, normalize=True)

In [98]:
cosine_metric2 = cosine(bag_of_words_n2)
euclidian_metric2 = euclidian(bag_of_words2, emails2, N2)
print(cosine_metric2.shape)
print(euclidian_metric2.shape)

(6750, 6750)
(6750, 6750)


Kolejne testy będę przeprowadzał już tylko dla metryki cosinusowej i euklidesowej

In [99]:
%%time
b_clusters = find_best_clusters(emails2, range(200, 3000, 100), euclidian_metric2)

CPU times: total: 1min
Wall time: 1min 7s


In [102]:
print_clusters_parameters(emails2, "Euklidesowa", b_clusters)

Użyta metryka Euklidesowa
Łącznie jest 2800 grup maili
Największy kluster składa się z 213 maili
Jest 1466 grup jednoelementowych
Kilka przykładowych klustrów

############################
SAME AS CONSIGNEE . . . . .


############################
LOG-WAY-SOLUTION SP. Z.O.O.- ATLASJANIKOWO, UL. GNIEZNIENSKA 67/69 62-006 KOBYLNICA POLAND PHONE: +48 (61) 663-6970   FAX: +48 (61) 663-6977

LOG-WAY-SOLUTION SP. Z.O.O.- ATLASJANIKOWO, UL. GNIEZNIENSKA 67/69 62-006 KOBYLNICA POLAND PHONE: +48 (61) 663-6970  FAX: +48 (61) 663-6977


############################
HELLMANN WORLDWIDE LOGISTICS (SHANGHAI) LTD.,GUANGZHOU BRANCH RM 2420-22 GUANGBAI XINYI BUILDING, NO.18-28, XIHU ROAD,  GUANGZHOU 510030,P.R.CHINA TEL:86 20 28330200 FAX:86 20

HELLMANN WORLDWIDE LOGISTICS (SHANGHAI) LTD.,GUANGZHOU BRANCH RM 2420-22 GUANGBAI XINYI BUILDING, NO.18-28, XIHU ROAD,  GUANGZHOU 510030,P.R.CHINA  TEL:86 20 28330200 FAX:86 2


############################
SPEDITION AUTO FREIGHT OY, KOTOLAHDENTIE 12, 48310 KOTK

In [103]:
%%time
b_clusters = find_best_clusters(emails2, range(200, 3000, 100), cosine_metric2)

CPU times: total: 1min 35s
Wall time: 1min 49s


In [106]:
print_clusters_parameters(emails2, "Cosinusowa", b_clusters)

Użyta metryka Cosinusowa
Łącznie jest 2800 grup maili
Największy kluster składa się z 40 maili
Jest 1121 grup jednoelementowych
Kilka przykładowych klustrów

############################
LLC FNS CIS SPB GAPSALSKAYA ST.5, BUSINESS CENTER BALTIC 4TH FLOOR, OFFICE 513, ZIP 198035 RUSSIA TEL. +7(812)680192

LLC FNS CIS SPB Gapsalskaya St.5 Business Center Baltic 4th floor, office 513


############################
TERMCOTANK Geneva RUE PEDRO MEYLAN,5 1208 GE, SWITZERLAND


############################
ERAS SPORTING CO., LTD. 1F.,NO.10ALLEY 1LANE 37ZHONGXIN ST., XINZHUANGCITY,TAIPEI COUNTY 242TAIWAN

ERAS SPORTING CO., LTD. 1F.,NO.10,ALLEY 1,LANE 37,ZHONGXIN ST.,XINZHUANG CITY,TAIPEI COUNTY 242,TAIWAN


############################
PANALPINA POLSKA SP. Z O.O.UL.17 STYCZNIA 52 PL 02-146 WARSZAWA / POLAND TEL: 48 22 609 65 30 FAX: 48 22609 65 91 AS AGENT FOR AND ON BEHALF OF PANTAINER(H.K.) LTD.

PANALPINA POLSKA SP.Z.O.O. AS AGENTFOR AND ON BEHALF OF PANTAINER LTDUL.17 STYCZNIA 52 WARSZAWA 0

## Wnioski

Użyty algorytm był uproszczoną wersją k-means. Klasteryzacja byłaby mogłaby być skuteczniejsza gdyby reprezentanci grup nie musieli należeć do zbiorów. Ponadto według mnie używanie metryk lcs, dice lub edit_distance jest w zadaniu klasteryzacji nieskuteczne, między innymi dlatego, że wektory muszą być wartościami naturalnymi, a także przez to jak długo zajmuje ich policzenie. 